Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [1]:
import retinoto_py as fovea

args = fovea.Params()
args

Running on MPS device (Apple Silicon/MacOS) - macos_version = 26.1
Random seed 1998 has been set.
Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	User laurentperrinet Working on host obiwan.local with device mps, pytorch==2.9.1


Params(image_size=224, num_epochs=5, n_train_stop=0, seed=1998, batch_size=250, verbose=True)

In [2]:
args

Params(image_size=224, num_epochs=5, n_train_stop=0, seed=1998, batch_size=250, verbose=True)

# testing each network on the validation dataset

In [13]:
import torch
import torchvision.models as models


VAL_DATA_DIR = args.DATAROOT / 'Imagenet_full' / 'train'

# --- 3. Load the Pre-trained ResNet Model ---
# We'll use ResNet50, a powerful and common choice.
# `pretrained=True` downloads the model weights trained on ImageNet.
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Move the model to the selected device (GPU or CPU)
model = model.to(args.device)

# Set the model to evaluation mode.
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
import torchvision.transforms as transforms
from PIL import Image
import os


In [ ]:
from retinoto_py import get_idx_to_label
idx_to_label = get_idx_to_label(args)

ImportError: cannot import name 'get_idx2label' from 'retinoto_py' (/Users/laurentperrinet/research/science/retinoto_py/src/retinoto_py/__init__.py)

In [6]:
# --- 5. Define Image Pre-processing ---
# The images must be pre-processed in the exact same way the model was trained on.
# This includes resizing, cropping, and normalizing.
preprocess = transforms.Compose([
    transforms.Resize(256),                # Resize the shortest side to 256px
    transforms.CenterCrop(224),            # Crop the center 224x224 pixels
    transforms.ToTensor(),                 # Convert the image to a PyTorch Tensor
    transforms.Normalize(                  # Normalize with ImageNet mean and std
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [20]:
from torchvision import datasets

# --- 2. Create Dataset and DataLoader using ImageFolder ---
# ImageFolder automatically infers class names from directory names
# and maps them to integer indices.
val_dataset = datasets.ImageFolder(root=VAL_DATA_DIR, transform=preprocess)

# The dataset provides a mapping from class index to class name (folder name)
class_to_idx = val_dataset.class_to_idx
# We often want the inverse mapping for printing results
idx_to_class = {v: k for k, v in class_to_idx.items()}

# The DataLoader handles batching, shuffling (for training), and loading data efficiently.
# For evaluation, we don't need to shuffle.
# A batch size of 1 is simplest for per-image analysis, but you can use larger batches.
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

# --- 3. Load the Model ---
# Make sure to load a model trained on the same number of classes
num_classes = len(val_dataset.classes)
num_ftrs = model.fc.in_features
print(f'Model has {num_ftrs} input features to final FC layer for {num_classes} classes.')

Model has 2048 input features to final FC layer for 1000 classes.


In [ ]:
# --- 4. Evaluation Loop ---
correct_predictions = 0
total_predictions = 0

print(f"Starting evaluation on {len(val_dataset)} images...")
print("-" * 50)

# We use tqdm to create a nice progress bar
for images, true_labels in tqdm(val_loader, desc="Evaluating"):
    # Move data to the correct device
    images = images.to(device)
    true_labels = true_labels.to(device)

    # Get predictions (no need for gradients)
    with torch.no_grad():
        outputs = model(images)
        _, predicted_labels = torch.max(outputs, 1)

    # --- Check for Correctness ---
    # The batch size is 1, so we get the first element of the tensors
    predicted_label_idx = predicted_labels[0].item()
    true_label_idx = true_labels[0].item()

    # Get the human-readable names (folder names)
    predicted_class_name = idx_to_class[predicted_label_idx]
    true_class_name = idx_to_class[true_label_idx]

    # Check if the prediction was correct
    is_correct = (predicted_label_idx == true_label_idx)
    if is_correct:
        correct_predictions += 1
    total_predictions += 1

    # Get the image filename for printing
    # The DataLoader doesn't easily give filenames, so we can't print them without more work.
    # For this example, we'll just print the class names.
    status = "✅ Correct" if is_correct else "❌ Incorrect"
    print(f"True: {true_class_name:<20} | Predicted: {predicted_class_name:<20} | {status}")

print("-" * 50)
accuracy = 100 * correct_predictions / total_predictions
print(f"Evaluation complete.")
print(f"Accuracy: {correct_predictions}/{total_predictions} ({accuracy:.2f}%)")



for image_file in image_files:

    try:
        # Load the image
        img = Image.open(image_file).convert('RGB')

        # Pre-process the image and add a batch dimension
        # PyTorch models expect a batch of images, so we unsqueeze a dimension.
        img_t = preprocess(img)
        batch_t = torch.unsqueeze(img_t, 0).to(args.device)

        # Run inference
        # `torch.no_grad()` tells PyTorch not to compute gradients, saving memory and computation
        with torch.no_grad():
            output = model(batch_t)

        # --- Get the Prediction ---
        # The output is a tensor of logits. We find the index of the highest logit.
        # `torch.max` returns the max value and its index. We only need the index.
        _, index = torch.max(output, 1)
        
        # Move the result to CPU and convert it to a Python integer
        predicted_index = index[0].item()
        
        # Get the human-readable label from our mapping
        predicted_label = idx2label[predicted_index]

        # --- Print the result ---
        print(f"Image: {image_file.name:<30} | Predicted: {predicted_label}")

    except Exception as e:
        print(f"Could not process {image_file.name}: {e}")

print("-" * 50)
print("Inference complete.")